# Manolo Ramírez Pintor - A01706155
## Módulo Big Data:
### Utilización, procesamiento y visualización de grandes volúmenes de datos

## 1. Configurando el entorno de trabajo PySpark

Iniciamos revisando los recursos que tenemos disponibles en el sistema

Tenemos aproximadamente medio GB de RAM utilizado de los 24 GB que tenemos disponibles en total, así que tenemos suficientes recursos para trabajar con un dataset grande.

In [1]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          23988       11522        6186        5024        6279        7165
Swap:             0           0           0


In [2]:
!whoami

ubuntu


Para evitar tener muchos mensajes de advertencia en el notebook, importamos warnings para filtrarlos todos

In [3]:
import warnings
warnings.filterwarnings('ignore')

Ahora, ponemos las rutas de donde tenemos instalado Java 8 y Spark

In [4]:
#Estableciendo variable de entorno
import os
# import pandas as pd
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-arm64"
os.environ["SPARK_HOME"] = "/home/mc/spark/spark-3.2.2-bin-hadoop3.2"

Importamos findspark e inicializamos la instalación de Spark

In [5]:
#Buscando e inicializando la instalación de Spark
import findspark
findspark.init()
findspark.find()

'/home/mc/spark/spark-3.2.2-bin-hadoop3.2'

Ahora importamos SparkSession y creamos una sesión para este trabajo, en mi caso se llama ``bigData_Manolo``

In [6]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder.appName('bigData_Manolo').getOrCreate()
spark_session

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/29 18:23:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 2. Seleccionando un dataset de gran tamaño
En mi caso, encontré y seleccioné el dataset de [Car Sales](https://www.kaggle.com/datasets/ekibee/car-sales-information), este lo encontré en Kaggle usando los filtros de tamaño. El peso del dataset descomprimido llega a 2GB.

## 3. Generando un modelo inteligente de regresión con MLlib
### a) EDA básico
Ya que en Kaggle se indica que los datos no están completamente limpios, voy a revisar qué columnas tienen datos faltantes y qué es lo que puede servirme para luego hacer ETL y comenzar a realizar predicciones.   

Al momento de cargar el dataset, podemos ver que existen distintas columnas. Estos son datos obtenidos de una página parecida a un Mercado Libre ruso de automóviles.   

Tenemos columnas como la marca, el modelo, el tipo de carrocería, color, tipo de combustible, año, kilometraje, tipo de transmisión, poder en caballos de fuerza, precio en rublos, el nombre del motor, la capacidad del motor (en litros), la fecha de publicación, la ubicación del automóvil, el link de la publicación, la descripción y el momento en el que se capturó la información desde la página.

In [7]:
df_spark = spark_session.read.option("header",True).csv('region25.csv')
df_spark.columns

['brand',
 'name',
 'bodyType',
 'color',
 'fuelType',
 'year',
 'mileage',
 'transmission',
 'power',
 'price',
 'vehicleConfiguration',
 'engineName',
 'engineDisplacement',
 'date',
 'location',
 'link',
 'description',
 'parse_date']

En base a la información que encontré en Kaggle, estableceremos el tipo de dato por columna.

In [8]:
df_spark = df_spark.withColumn("date",df_spark.date.cast('string'))
df_spark = df_spark.withColumn("parse_date",df_spark.parse_date.cast('string'))
df_spark = df_spark.withColumn("year",df_spark.year.cast('int'))
df_spark = df_spark.withColumn("mileage",df_spark.mileage.cast('int'))
df_spark = df_spark.withColumn("power",df_spark.power.cast('int'))
df_spark = df_spark.withColumn("price",df_spark.price.cast('int'))

A continuación realizaré un describe, aparecerá roto por el gran número de columnas pero lo arreglaré con Markdown...

* Tenemos 1,513,200 filas totales en el dataset.   
* El precio promedio de los automóviles es de 1,368,558.3 rublos.   
* Hay columnas que son de datos numéricos pero que presentan valores nulos.

In [9]:
df_spark.describe().show()

+-------+-------+------------------+-------------+-------+--------+------------------+------------------+------------+-----------------+------------------+--------------------+-----------+------------------+-------------------+-----------+--------------------+--------------------+--------------------+
|summary|  brand|              name|     bodyType|  color|fuelType|              year|           mileage|transmission|            power|             price|vehicleConfiguration| engineName|engineDisplacement|               date|   location|                link|         description|          parse_date|
+-------+-------+------------------+-------------+-------+--------+------------------+------------------+------------+-----------------+------------------+--------------------+-----------+------------------+-------------------+-----------+--------------------+--------------------+--------------------+
|  count|1513200|           1513200|      1513200|1403466| 1509640|           1102226|     

Ahora veremos si las columnas tienen el tipo de dato correcto.

In [10]:
# Obtenemos el tipo de dato de las columnas
for col in df_spark.dtypes:
    print(col[0]+" , "+col[1])

brand , string
name , string
bodyType , string
color , string
fuelType , string
year , int
mileage , int
transmission , string
power , int
price , int
vehicleConfiguration , string
engineName , string
engineDisplacement , string
date , string
location , string
link , string
description , string
parse_date , string


Ya que ahora los tipos de dato son correctos, procederemos a ver qué columnas presentan valores nulos

In [11]:
# Ahora contaremos los valores nulos con isnan, when, count y col
from pyspark.sql.functions import isnan, when, count, col
df_spark.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_spark.columns]).show(truncate=False)

+-----+----+--------+------+--------+------+-------+------------+-----+-----+--------------------+----------+------------------+----+--------+----+-----------+----------+
|brand|name|bodyType|color |fuelType|year  |mileage|transmission|power|price|vehicleConfiguration|engineName|engineDisplacement|date|location|link|description|parse_date|
+-----+----+--------+------+--------+------+-------+------------+-----+-----+--------------------+----------+------------------+----+--------+----+-----------+----------+
|0    |0   |0       |109734|3560    |410974|14480  |3065        |20887|0    |410974              |412058    |420765            |0   |0       |0   |35737      |0         |
+-----+----+--------+------+--------+------+-------+------------+-----+-----+--------------------+----------+------------------+----+--------+----+-----------+----------+



Como la tabla no sale bien, pondré manualmente qué columnas presentan valores nulos y cuántos:
* Color: 109,734
* fuelType: 3,560
* Year: 410,974
* Mileage: 14,480
* Transmission: 3,065
* Power: 20,887
* vehicleConfiguration: 410,974
* engineName: 412,058
* engineDisplacement: 420,765
* description: 35,737

In [12]:
# Ahora para obtener los valores únicos, utilizaremos count_distincst
from pyspark.sql.functions import count_distinct

# Contaremos los valores únicos de cada columna
df_spark.select([count_distinct(c).alias(c) for c in df_spark.columns]).show()

+-----+----+--------+-----+--------+----+-------+------------+-----+-----+--------------------+----------+------------------+----+--------+-----+-----------+----------+
|brand|name|bodyType|color|fuelType|year|mileage|transmission|power|price|vehicleConfiguration|engineName|engineDisplacement|date|location| link|description|parse_date|
+-----+----+--------+-----+--------+----+-------+------------+-----+-----+--------------------+----------+------------------+----+--------+-----+-----------+----------+
|   74|1026|      11|   16|       3|  59|    541|           5|  352| 2986|                7955|      1149|                55|  39|      71|50119|      63606|      2521|
+-----+----+--------+-----+--------+----+-------+------------+-----+-----+--------------------+----------+------------------+----+--------+-----+-----------+----------+



Como la tabla no sale bien, pondré manualmente los valores únicos de cada columna:
* brand:	67
* name:	884
* bodyType:	11
* color:	16
* fuelType:	3
* year:	54
* mileage:	483
* transmission:	5
* power:	323
* price:	2037
* vehicleConfiguration:	5620
* engineName:	897
* engineDisplacement:	53
* date:	16
* location:	69
* link:	24757
* description:	28790
* parse_date:	307

### b) ETL
Viendo un poco las columnas que tenemos y las descripciones cortas obtenidas de Kaggle, tenemos información que **no nos va a servir de inicio**, como ``parse_date``, ``description``, ``link``, ``location``, ``date``, ``engineDisplacement``, ``engineName``, ``vehicleConfiguration`` y ``year``.   

La justificación de quitarlos es que presentan una gran cantidad de datos nulos, no son relevantes para la venta de un automóvil, existen automóviles viejos que pueden ser muy baratos y muy caros a la vez e incluso hay datos que son únicos por registro. Entonces procederemos a hacer un drop de las columnas.

In [13]:
df_spark = df_spark.drop('year','vehicleConfiguration','engineName',
                         'engineDisplacement', 'link','description',
                         'parse_date','date','location')

Revisamos las columnas que nos quedan, ahora podremos trabajar mejor con estos datos.

In [14]:
df_spark.columns

['brand',
 'name',
 'bodyType',
 'color',
 'fuelType',
 'mileage',
 'transmission',
 'power',
 'price']

Procederemos a revisar las columnas con pocos valores únicos y que coincidan con los nulos para ver lo que contienen

In [15]:
# Obtenemos los colores de los automóviles
df_spark.select('color').distinct().collect()

[Row(color='Бордовый'),
 Row(color='Белый'),
 Row(color='Золотистый'),
 Row(color='Коричневый'),
 Row(color=None),
 Row(color='Оранжевый'),
 Row(color='Серебристый'),
 Row(color='Розовый'),
 Row(color='Фиолетовый'),
 Row(color='Бежевый'),
 Row(color='Красный'),
 Row(color='Голубой'),
 Row(color='Серый'),
 Row(color='Желтый'),
 Row(color='Зеленый'),
 Row(color='Черный'),
 Row(color='Синий')]

In [16]:
# Obtenemos los tipos de combustión de los automóviles
df_spark.select('fuelType').distinct().collect()

[Row(fuelType=None),
 Row(fuelType='Дизель'),
 Row(fuelType='Электро'),
 Row(fuelType='Бензин')]

In [17]:
# Transmisión
df_spark.select('transmission').distinct().collect()

[Row(transmission='Механика'),
 Row(transmission='Робот'),
 Row(transmission=None),
 Row(transmission='Вариатор'),
 Row(transmission='Автомат'),
 Row(transmission='АКПП')]

Después de un análisis rápido, pude ver que es posible generalizar datos nulos en vez de borrarlos directamente y comenzar a perder información.   

Por ejemplo, el color más común de los automóviles es blanco y podemos partir de ahí para rellenar los strings nulos o vacíos con ``Белый``.

In [18]:
df_spark = df_spark.na.fill('Белый', 'color')

Ahora, el tipo de combustible que más se ues es la gasolina, siendo la más común de los automóviles, así que llenaré el tipo de combustible con ``Бензин``.

In [19]:
df_spark = df_spark.na.fill('Бензин', 'fuelType')

Revisando el progreso de los valores nulos, ahora sólo quedan las columnas de kilometraje, poder y transmisión.

In [20]:
df_spark.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_spark.columns]).show(truncate=False)

+-----+----+--------+-----+--------+-------+------------+-----+-----+
|brand|name|bodyType|color|fuelType|mileage|transmission|power|price|
+-----+----+--------+-----+--------+-------+------------+-----+-----+
|0    |0   |0       |0    |0       |14480  |3065        |20887|0    |
+-----+----+--------+-----+--------+-------+------------+-----+-----+



Las columnas de poder y kilometraje las podemos reemplazar con valores de la media, el promedio o la moda ya que son columnas con el tipo de dato entero.

In [21]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols = ['power', 'mileage'],
    outputCols = ['power', 'mileage']
).setStrategy('mean')

In [22]:
df_spark = imputer.fit(df_spark).transform(df_spark)

Quizá sea bueno poner los tipos de transmisión faltantes con la moda por esta ocasión.   

Usaré consultas SQL ya que no encontré otra manera de hallar la moda y el Imputer tristemente me da error si trato de hacerlo con tipo de dato de string.

In [23]:
df_aux = df_spark.where(col('transmission').isNotNull())

df_aux.createOrReplaceTempView('table')
df_aux_2 = spark_session.sql(
    'SELECT transmission, COUNT(transmission) AS count FROM table GROUP BY transmission ORDER BY count desc'
)

df_aux_2.show()

+------------+------+
|transmission| count|
+------------+------+
|    Вариатор|677023|
|        АКПП|645355|
|    Механика| 73331|
|     Автомат| 59233|
|       Робот| 55193|
+------------+------+



Podemos observar que el tipo de transmisión más común es el de CVT, (o transmisión continuamente variable), así que se lo asignaremos a los valores nulos.

In [24]:
df_spark = df_spark.na.fill('Вариатор', 'transmission')

Ahora revisaré si ya no tenemos valores nulos en nuestro dataset...

In [25]:
df_spark.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_spark.columns]).show(truncate=False)

+-----+----+--------+-----+--------+-------+------------+-----+-----+
|brand|name|bodyType|color|fuelType|mileage|transmission|power|price|
+-----+----+--------+-----+--------+-------+------------+-----+-----+
|0    |0   |0       |0    |0       |0      |0           |0    |0    |
+-----+----+--------+-----+--------+-------+------------+-----+-----+



Ya que no tenemos ningún dato nulo y puedo decir que ahora mis datos están limpios, guardaré una copia y la insertaré dentro de Tableau más adelante para observar datos de forma visual y darme una idea de cómo están las cosas. 😀   


In [26]:
df_spark.repartition(1).write.option("header", True).csv('region25_noNull.csv')

### c) Generando el modelo con MLLib
Ahora vamos a generar un modelo inteligente de clasificación de regresión con el objetivo de predecir el precio de los automóviles en el mercado libre ruso de automóviles.   

Como tengo datos en string, es importante que convierta esa información a datos categóricos. El aprendizaje de máquina no funciona si le das strings así que tiene que existir una conversión numérica.   

El ``[:-1]`` sirve para excluír la última columna que queremos predecir (precio), así que no saldrá en el output

In [27]:
categorical_cols = [item[0] for item in df_spark.dtypes if item[1].startswith('string')]
print(categorical_cols)

numerical_cols = [item[0] for item in df_spark.dtypes if item[1].startswith('int') | item[1].startswith('double')][:-1]
print(numerical_cols)

['brand', 'name', 'bodyType', 'color', 'fuelType', 'transmission']
['mileage', 'power']


Podemos observar que tenemos 6 variables categóricas y 2 variables numéricas.   

Ahora sí para convertir los datos vamos a usar las siguientes herramientas de PySpark:

In [28]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
stages = [] # Transformer / Estimator

# Convertir de strings a 0, 1, 2, etc. (Del más frecuente al menos)
# Se creará en una nueva columna
for categoricalCol in categorical_cols:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    OHencoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "_catVec"])
    stages += [stringIndexer, OHencoder]
assemblerInputs = [c + "_catVec" for c in categorical_cols] + numerical_cols
Vectassembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [Vectassembler]

Ya que convertimos nuestras variables categóricas a numéricas, aplicaremos los stages de la transformación que hicimos arriba para obtener los features que queremos usar para el aprendizaje de **regresión multilineal.**   

Esto lo haremos con ayuda del pipeline de MLLib, así transformaremos algunos datos.

In [29]:
from pyspark.ml import Pipeline
# import pandas as pd
cols = df_spark.columns
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df_spark)
df_spark = pipelineModel.transform(df_spark)
selectedCols = ['features']+cols
df_spark = df_spark.select(selectedCols)
# pd.DataFrame(df_spark.take(5), columns=df_spark.columns)

df_spark.show()

+--------------------+-------------+------------------+-------------+--------+--------+-------+------------+-----+-------+
|            features|        brand|              name|     bodyType|   color|fuelType|mileage|transmission|power|  price|
+--------------------+-------------+------------------+-------------+--------+--------+-------+------------+-----+-------+
|(1131,[52,697,111...|         Fiat|        124 Spider|     Открытый|   Синий|  Бензин|   8000|     Автомат|  145|1830000|
|(1131,[10,624,109...|          BMW|                i3|Хэтчбек 5 дв.|  Черный| Электро|  12000|     Автомат|  145|1830000|
|(1131,[8,205,1098...|Mercedes-Benz|         GLE Coupe|   Джип 5 дв.|Бордовый|  Бензин|  57000|        АКПП|  367|4600000|
|(1131,[8,179,1098...|Mercedes-Benz|           G-Class|   Джип 5 дв.|  Черный|  Бензин| 200000|        АКПП|  296|2999999|
|(1131,[12,212,109...|         Audi|                Q7|   Джип 5 дв.|   Белый|  Бензин|  67000|     Автомат|  252|3300000|
|(1131,[3,120,10

Ya que tenemos las dos columnas que nos importan usar para el entrenamiento y pruebas, haremos un select de ambas

In [30]:
df_spark_final = df_spark.select("features", "price")

df_spark_final.show(5)

+--------------------+-------+
|            features|  price|
+--------------------+-------+
|(1131,[52,697,111...|1830000|
|(1131,[10,624,109...|1830000|
|(1131,[8,205,1098...|4600000|
|(1131,[8,179,1098...|2999999|
|(1131,[12,212,109...|3300000|
+--------------------+-------+
only showing top 5 rows



Ahora separamos los datos de entrenamiento y de prueba, daré 80% de datos para entrenar y 20% de datos para probar el modelo.

In [42]:
train, test = df_spark_final.randomSplit([0.8, 0.2])

Procedemos a usar la función de regresión linear con gradiente descendiente usando los datos de entrenamiento.

In [43]:
from pyspark.ml.regression import LinearRegression

# Creamos el modelo en una variable y le asignamos las columnas para entrenar
MLR = LinearRegression(featuresCol="features", labelCol="price")

Ahora, haremos el entrenamiento del modelo usando la función fit()

In [44]:
modelo = MLR.fit(train)

22/10/29 18:30:58 WARN Instrumentation: [ea04eda5] regParam is zero, which might cause numerical instability and overfitting.
22/10/29 18:31:12 WARN Instrumentation: [ea04eda5] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


Podemos observar que se está haciendo el fit aunque salga esa advertencia.   

La advertencia sale ya que MLLib está a punto de ser descontinuada ya que las herramientas ML de PySpark son mejores en eficiencia y uso.

## 4. Evaluando el modelo con PySpark

Ya que el entrenamiento ha finalizado, ahora haremos una evaluación usando las herramientas nativas de PySpark

In [45]:
prediccion = modelo.evaluate(train)

Ahora mostraremos las predicciones a ojo de buen cubero para ver qué tan acercado es el modelo a los datos reales, usando el método de predictions

In [46]:
prediccion.predictions.show()

+--------------------+-------+------------------+
|            features|  price|        prediction|
+--------------------+-------+------------------+
|(1131,[0,73,1098,...|4070000|3308410.5401063035|
|(1131,[0,73,1098,...|4070000|3308410.5401063035|
|(1131,[0,73,1098,...|4070000|3308410.5401063035|
|(1131,[0,73,1098,...|4070000|3308410.5401063035|
|(1131,[0,73,1098,...|4070000|3308410.5401063035|
|(1131,[0,73,1098,...|4070000|3308410.5401063035|
|(1131,[0,73,1098,...|4070000|3308410.5401063035|
|(1131,[0,73,1098,...|4070000|3308410.5401063035|
|(1131,[0,73,1098,...|4070000|3308410.5401063035|
|(1131,[0,73,1098,...|4070000|3308410.5401063035|
|(1131,[0,73,1098,...|4070000|3308410.5401063035|
|(1131,[0,73,1098,...|4250000|3308410.5401063035|
|(1131,[0,73,1098,...|4250000|3308410.5401063035|
|(1131,[0,73,1098,...|4250000|3308410.5401063035|
|(1131,[0,73,1098,...|4250000|3308410.5401063035|
|(1131,[0,73,1098,...|4250000|3308410.5401063035|
|(1131,[0,73,1098,...|4250000|3308410.5401063035|


Podemos observar que la predicción no es muy acertada, vamos a mostrar los coeficientes que se generaron con nuestro modelo:

In [47]:
coefficient = modelo.coefficients
print(coefficient)

[51934.65907005527,-185390.13881492024,-238029.53564804254,485658.10849771847,-204724.37843062548,-20876.894965088828,6503.025215926085,-99946.26898600116,627705.3820627648,-92440.34989109801,475830.07931478834,827384.4047355,-23269.620389467436,-75091.65102671973,-162606.57346635827,195471.88751269237,468881.3849526611,105994.54629031259,-135906.04428211812,101768.79856994275,-297978.26835997915,-87062.31151841398,-179855.3789220619,668997.7079297022,-554662.0206524454,146717.1302459972,1765780.95992784,-135754.559138954,201711.95323831742,-296541.9978342758,1965131.6541113537,182848.5538388046,-395338.744608471,543980.3691963995,2496719.21908483,451740.79931728763,80046.31508045243,-193081.23499151607,1175349.3319867002,-456981.38566286996,-629545.2700895299,-289612.05952305684,-280936.74163441034,-660697.0504242165,841844.0067549422,696740.4930025802,-446852.8657864232,-810380.9507097952,10026.217805833225,-354524.5544838656,-594994.8178969088,-353999.02961797523,-189769.0024075005,

Ahora mostraremos el intercept value de nuestro modelo

In [48]:
intercept = modelo.intercept
print(intercept)

971689.1524341


Ahora, evaluaremos el modelo usando la función de R2 que tiene incluída PySpark ML

In [49]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluation = RegressionEvaluator(labelCol="price", predictionCol="prediction")

r2 = evaluation.evaluate(prediccion.predictions, {evaluation.metricName: "r2"})
print("r2: %.3f" %r2)

r2: 0.718


Podemos observar que **nuestro modelo tiene un r2 de 71%** respecto a los datos que ya hemos visto, no está muy mal pero podría ser mejor.   

Ahora probaremos con los datos que no ha visto el modelo antes:

In [50]:
test_prediction = modelo.evaluate(test)

Ahora a ojo de buen cubero, vamos a ver las nuevas predicciones

In [51]:
test_prediction.predictions.show()

+--------------------+-------+------------------+
|            features|  price|        prediction|
+--------------------+-------+------------------+
|(1131,[0,73,1098,...|4070000|3308410.5401063035|
|(1131,[0,73,1098,...|4250000|3308410.5401063035|
|(1131,[0,73,1098,...|4250000|3308410.5401063035|
|(1131,[0,73,1098,...|4075000|3304255.7425589683|
|(1131,[0,73,1098,...|4075000|3304255.7425589683|
|(1131,[0,73,1098,...|4075000|3304255.7425589683|
|(1131,[0,73,1098,...|4075000|3304255.7425589683|
|(1131,[0,73,1098,...|4075000|3304255.7425589683|
|(1131,[0,73,1098,...|4999999|3304255.7425589683|
|(1131,[0,73,1098,...|4999999|3304255.7425589683|
|(1131,[0,73,1098,...|4999999|3304255.7425589683|
|(1131,[0,73,1098,...|4999999|3304255.7425589683|
|(1131,[0,73,1098,...|4075000| 3300100.945011633|
|(1131,[0,73,1098,...|4075000| 3300100.945011633|
|(1131,[0,73,1098,...|4075000| 3300100.945011633|
|(1131,[0,73,1098,...|3565000|3295946.1474642972|
|(1131,[0,73,1098,...|3565000|3295946.1474642972|


Ahora obtendremos el r2 con los resultados de los datos que no vio el modelo:

In [52]:
evaluation = RegressionEvaluator(labelCol="price", predictionCol="prediction")

r2 = evaluation.evaluate(test_prediction.predictions, {evaluation.metricName: "r2"})
print("r2: %.3f" %r2)

r2: 0.721


Podemos observar que la predicción con los datos de prueba es de 72%, es mejor que lo que teníamos con los datos de entrenamiento, esto quiere decir que nuestro modelo presenta **underfitting**, aunque viendo las predicciones tenemos **varianza baja**. 😄

## 5. Generando un tablero de visualización con Tableau

Para crear el tablero de visualización, usaré el dataset arreglado de valores nulos, así no tendré información que falta y podré ver también cómo están pasando las cosas dentro del dataset. No importa si son datos grandes, la visualización nos ayudará a comprender mejor en vez de observar sólo números.   

La herramienta a utilizar será Tableau.

Haciendo varias combinaciones del precio respecto a otras columnas pude percatarme de que había algo que no tenía relación directa con el precio...   

Ahora mostraremos nuestro primer gráfico

### Precio vs marca y millas recorridas
![kilometraje vs precio](images/kilometraje_vs_precio.png)
En este gráfico podemos observar que las millas recorridas tienen cierta relación por el precio de los automóviles, en el dataset, los automóviles que son de marcas menos reconocidas que las más reconocidas, tienden a tener mayor kilometraje y su precio se reduce más.

### Precio vs marca y caballos de fuerza
![Precio vs marca y caballos de fuerza](images/caballos_vs_precio.png)
En este gráfico podemos observar que los precios aumentan entre más caballos de fuerza tengan los automóviles, claramente con marcas como Lamborghini siento el top.

### Precio vs marca, caballos de fuerza y transmisión
![Precio vs marca, caballos de fuerza y transmisión](images/precio_vs_marca_caballos_transmision.png)
En este gráfico podemos observar que la transmisión también juega un papel en el aumento del precio de los automóviles.

### Precio marca y modelo
![Precio vs marca, caballos de fuerza y transmisión](images/precio_vs_marca_y_modelo.png)
En este gráfico podemos observar que el precio depende mucho de la marca pero al mismo tiempo del modelo que estás adquiriendo, ya que existen mejores modelos que otros, lo cual hace que cambien sus precios de venta. 

### Tipo de combustible vs la mediana del precio
![Tipo de combustible vs la mediana del precio](images/combustible_vs_medianPrice.png)
En este gráfico podemos observar que el precio por el tipo de combustible es mayor en el diesel, convirtiéndolo en el tipo de automóvil más caro por combustión, después sigue el eléctrico y al final la gasolina.   

Esto me hace pensar que como el automóvil de gasolina se vende más barato que uno eléctrico, sigue siendo la opción principal de las personas al momento de su compra.

### Tipo de carrocería vs precio
![Tipo de carrocería vs precio](images/carroceria_vs_precio.png)
Finalmente, en este gráfico podemos observar que el tipo de carrocería es un poco representativo en el precio, ya que existen muchos tipos y el precio más alto aplica en sólo las que se consideran de lujo o especiales.

Por el momento es todo el análisis que se me ocurrió hacer y espero sea suficiente. Concluyo que quizá cambiando o quitando columnas pueda mejorar el resultado de predicción, o bien, usando otro modelo para realizar las predicciones del precio de los automóviles.   

Me siento satisfecho por el trabajo realizado, ¡muchas gracias! 😊.